In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from random import randint

import matplotlib.pyplot as plt
%matplotlib inline

batch_size = 25
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.ToTensor()),
        batch_size = 10000, shuffle=True)

from random import randint

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

num_classes=10

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=3, dropout=0):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.gru = nn.GRU(input_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        self.hidden = None
    def forward(self, input_seqs, input_lengths):

        input_seqs = input_seqs.unsqueeze(-1).type(torch.FloatTensor)
        packed = torch.nn.utils.rnn.pack_padded_sequence(input_seqs, input_lengths)
        outputs, self.hidden = self.gru(packed, self.hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) 
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] 
        
        return outputs

In [3]:
class CircuitNet(nn.Module):
    def __init__(self,num_classes=12,num_lstm_layers=3):
        super(CircuitNet, self).__init__()
        
        
        self.state_size = 100
        self.num_lstm_layers = num_lstm_layers
        
        #monitor use of each circuit to balance training
        self.used_circuits = np.zeros([40,2])
        for i in range(40):
            self.used_circuits[i][0] = i
        
        
        
        self.circuits_to_use = 5
        self.n_circuits = 10
        self.circuit_in_dim = 4
        #self.circuit_in_x = 2
        #self.circuit_in_y = 2
        
        #circuit picker
        self.circuit_picker = nn.Sequential(
            nn.Conv1d(1, 16, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2, stride=2),  
            nn.Conv1d(16, 8, 3, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(8),
            nn.MaxPool1d(2, stride=2)
        )
        
        self.circuit_picker_out = nn.LSTM(1568,(self.n_circuits *  self.circuits_to_use), self.num_lstm_layers)
        self.hidden = None
        #self.circuit_picker_out = nn.Linear(8*7*7, (self.n_circuits *  self.circuits_to_use))
        
        #data pipe
        self.data_pipe = nn.Sequential(
            nn.Conv1d(1, 4, 4, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(4),
            nn.Conv1d(4, 8, 4, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(8),
            nn.MaxPool1d(2, stride=2), 
            
            
            nn.Conv1d(8, 4, 4, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(4),
            nn.Conv1d(4, 8, 4, stride=1, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(8),
            nn.MaxPool1d(2, stride=2)
        )
        self.data_pipe_out = nn.Linear(1560, (self.circuits_to_use * self.circuit_in_dim))
        
            
        self.circuits = []
        #define circuits
        for i in range(self.n_circuits):
        
            
            self.circuits.append(nn.Sequential(
                nn.Conv1d(1, 8, kernel_size=2, stride=1, padding=1),
                nn.BatchNorm1d(8),
                nn.ReLU(True),
                #nn.MaxPool2d(kernel_size=2, stride=2)))
            
                nn.Conv1d(8, 16, kernel_size=2, stride=1, padding=1),
                nn.BatchNorm1d(16),
                nn.ReLU(True),
                
                nn.MaxPool1d(kernel_size=2, stride=2)))
        
        #connect circuits (this is temporary)
        self.connect = nn.Sequential(
            nn.Linear(1440, 28*28))
        
        #final circuit to prediction
        self.output = nn.Sequential(
            #nn.Linear(640, 400),
            nn.Linear(240, num_classes))
        
    #init hidden state for lstm layer
    def init_hidden(self):
        return (torch.zeros(self.num_lstm_layers,1, (self.n_circuits *  self.circuits_to_use), device=device),
                torch.zeros(self.num_lstm_layers,1, (self.n_circuits *  self.circuits_to_use), device=device))
    
    def connection(self,x,verbose,circuit_dropout,circuit_ban_rate):
        x_flat = x.view(x.size(0),x.size(1),x.size(2)*x.size(3))
        circuit_pick = self.circuit_picker(x_flat)
        circuit_pick = circuit_pick.view(circuit_pick.size(0),1, -1)
        
        circuit_pick, self.hidden = self.circuit_picker_out(circuit_pick,self.hidden)
        
        
        circuit_pick = circuit_pick.view(circuit_pick.size(0),self.circuits_to_use,self.n_circuits)
        circuit_pick = circuit_pick.max(2)[1]
        
        data_pipe_in = torch.cat((circuit_pick.unsqueeze(1).type(torch.cuda.FloatTensor),x_flat),2)
        #circuit_pick_padded = F.pad(circuit_pick,(0,28-circuit_pick.size(1))).unsqueeze(1).unsqueeze(1)
        #circuit_pick_padded = circuit_pick_padded.type(torch.cuda.FloatTensor)
        #data_pipe_in = torch.cat((circuit_pick_padded,x),2)
        #data_pipe_in = data_pipe_in.view(data_pipe_in.size(0),data_pipe_in.size(1),
        #                                 data_pipe_in.size(2)*data_pipe_in.size(3))
        
        data_pipe = self.data_pipe(data_pipe_in)

        data_pipe = data_pipe.view(data_pipe.size(0), -1)
        data_pipe = self.data_pipe_out(data_pipe)
        data_pipe = data_pipe.view(data_pipe.size(0),self.circuits_to_use,self.circuit_in_dim)
        bans = np.random.randint(self.n_circuits,size=int(circuit_ban_rate*self.n_circuits))
        circuit_out = []
        for i in range(self.circuits_to_use):
            circuit_idx = circuit_pick[:,i]
            if(verbose):
                print(str(circuit_idx.cpu().numpy()))
                
            batch = []
            for b in range(batch_size):
                #print(data_pipe[b][i].shape)
                data = data_pipe[b][i]#.view(data_pipe[b][i].size(0)*data_pipe[b][i].size(1))
                data = data.unsqueeze(0).unsqueeze(0)
                circuit = self.circuits[circuit_idx[b]](data)
                #apply dropout
                if randint(1,100) < circuit_dropout*100 or circuit_idx[b] in bans:
                    circuit = torch.zeros(circuit.shape,device=device)  
                
                batch.append(circuit)
            batch = torch.cat(batch,0)
            circuit_out.append(batch)
        circuit_out = torch.cat(circuit_out,1)
        return circuit_out
    
    def forward(self, x, verbose,circuit_dropout=0,circuit_ban_rate = 0):
        
        circuit_out1 = self.connection(x,verbose,circuit_dropout,circuit_ban_rate)
        circuit_out_flat1 = circuit_out1.view(circuit_out1.size(0), -1)
        
        #connect = self.connect(circuit_out_flat1)
        #connect = connect.view(connect.size(0),1,28,28)
        
       # circuit_out2 = self.connection(connect,verbose)
        #circuit_out_flat2 = circuit_out2.view(circuit_out2.size(0), -1)
        out = self.output(circuit_out_flat1)
        
        return out
    
        

if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
model = CircuitNet().to(device)
torch.set_default_tensor_type('torch.FloatTensor')

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device)[0:batch_size], target.to(device)[0:batch_size]
    output = model(data,False)
    break

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
iter_print = 50

for batch_idx, (data_, target_) in enumerate(train_loader):
    #print(target)
   
    
    
    #train on digits 1-6, then train on 7-9 to check adaptability
    for i in range(1000):
        try:
            rand = randint(0, 3000)
            indices = np.where(target_<6)[0][rand:rand+batch_size]
            #indices = 
            data = np.take(data_,indices,axis = 0)
            target = np.take(target_,indices)

           # print(np.where(target_<6)[0][0:batch_size])
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            verbose = False
            if(i % iter_print == 0):
                verbose = True
            output = model(data,verbose,circuit_dropout=0.5,circuit_ban_rate = 0.2)
            loss =  nn.CrossEntropyLoss()(output, target)
            #loss = (output-target)**2
            loss.backward()
            optimizer.step()

            model.hidden = None
            if(i % iter_print == 0):
                print("Index : {}  Loss : {:.5f}  Real : {}  Pred : {}".format(i,
                                                                               loss[0],target[0:4].cpu().numpy(),
                                                                               output[0:4].max(1)[1].cpu().numpy()))
        except:
            continue
    print("switch")
    for i in range(500):
        try:
            rand = randint(0, 3000)
            indices = np.where(target_>6)[0][rand:rand+batch_size]
            #indices = 
            data = np.take(data_,indices,axis = 0)
            target = np.take(target_,indices)

           # print(np.where(target_<6)[0][0:batch_size])
            data, target = data.to(device), target.to(device)

            optimizer2.zero_grad()
            verbose = False
            if(batch_idx % iter_print == 0):
                verbose = False
            output = model(data,verbose)
            loss =  nn.CrossEntropyLoss()(output, target)
            #loss = (output-target)**2
            loss.backward()
            optimizer2.step()

            model.hidden = None
            if(i % 1 == 0):
                print("Index : {}  Loss : {:.5f}  Real : {}  Pred : {}".format(i,
                                                                               loss[0],target[0:4].cpu().numpy(),
                                                                               output[0:4].max(1)[1].cpu().numpy()))
        except:
            continue

[3 3 3 3 3 3 3 3 3 3 3 7 7 3 3 3 3 3 3 3 3 3 3 3 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[1 8 1 8 1 8 1 1 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[7 7 7 7 5 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
Index : 0  Loss : 2.59900  Real : [1 5 1 3]  Pred : [2 2 5 2]


/home/liam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[7 7 3 3 3 3 3 7 7 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[0 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 0 0 0 0 0 8 8 8 8]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 1 8 8 8 1 8 8 8 8 8]
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 9]
Index : 50  Loss : 1.64378  Real : [4 5 2 3]  Pred : [4 4 3 3]
[7 7 7 7 7 7 7 3 3 3 3 3 3 3 3 3 3 7 7 7 7 7 7 3 3]
[0 0 0 0 8 8 8 8 8 0 0 0 0 8 8 8 0 8 8 8 0 0 8 8 8]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[7 7 9 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
Index : 100  Loss : 1.33353  Real : [5 1 5 1]  Pred : [5 3 4 1]
[7 7 7 7 3 3 3 3 7 3 7 7 3 3 3 3 3 3 3 3 3 3 3 3 3]
[0 0 8 8 8 8 8 8 8 8 8 8 0 0 0 8 8 8 8 8 8 8 8 0 8]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[7 7 7 7 7 7 7 7 7 9 9 9 3 7 7 7 7 7 7 7 7 7 7 7 7]
Index : 150  Loss : 1.65622  Real : [5 2 5 1]  Pred : [3 4 4 1]
[7 1 3 3 3 3 3 3 3 3 3 3 3 3 

/home/liam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Index : 1  Loss : 19.36314  Real : [7 7 9 8]  Pred : [2 3 4 0]
Index : 2  Loss : 18.16372  Real : [7 7 7 9]  Pred : [4 4 1 4]
Index : 3  Loss : 18.70539  Real : [8 7 7 7]  Pred : [5 4 3 3]
Index : 4  Loss : 17.56542  Real : [7 7 9 7]  Pred : [3 3 3 3]
Index : 5  Loss : 17.09635  Real : [7 8 8 7]  Pred : [0 5 3 3]
Index : 6  Loss : 16.62674  Real : [8 8 8 7]  Pred : [3 3 0 3]
Index : 7  Loss : 15.77961  Real : [8 9 8 7]  Pred : [0 4 3 3]
Index : 8  Loss : 15.38437  Real : [9 9 9 7]  Pred : [4 4 3 3]
Index : 9  Loss : 14.49532  Real : [9 7 8 8]  Pred : [3 4 0 3]
Index : 10  Loss : 14.26963  Real : [9 7 8 9]  Pred : [4 3 0 3]
Index : 11  Loss : 14.25611  Real : [8 7 7 7]  Pred : [0 4 3 3]
Index : 12  Loss : 13.67064  Real : [8 7 9 8]  Pred : [3 3 3 3]
Index : 13  Loss : 13.19478  Real : [9 9 7 8]  Pred : [4 0 3 3]
Index : 14  Loss : 13.23779  Real : [9 8 9 7]  Pred : [3 0 4 3]
Index : 15  Loss : 12.91413  Real : [8 7 8 9]  Pred : [5 3 5 4]
Index : 16  Loss : 12.41157  Real : [9 9 7 8]  Pr

...

Index : 483  Loss : 0.01647  Real : [7 9 8 7]  Pred : [7 9 8 7]
Index : 484  Loss : 0.04958  Real : [7 9 7 9]  Pred : [7 9 7 9]
Index : 485  Loss : 0.21937  Real : [7 7 9 9]  Pred : [7 7 9 9]
Index : 486  Loss : 0.03237  Real : [8 8 8 8]  Pred : [8 8 8 8]
Index : 487  Loss : 0.05193  Real : [8 9 9 8]  Pred : [8 9 9 8]
Index : 488  Loss : 0.01788  Real : [8 7 8 8]  Pred : [8 7 8 8]
Index : 489  Loss : 0.03906  Real : [8 9 8 8]  Pred : [8 9 8 8]
Index : 490  Loss : 0.12903  Real : [8 8 7 9]  Pred : [8 8 7 9]
Index : 491  Loss : 0.17946  Real : [9 9 7 9]  Pred : [9 9 7 9]
Index : 492  Loss : 0.02917  Real : [9 7 7 8]  Pred : [9 7 7 8]
Index : 493  Loss : 0.01888  Real : [9 9 7 9]  Pred : [9 9 7 9]
Index : 494  Loss : 0.04007  Real : [9 8 8 7]  Pred : [9 8 8 7]
Index : 495  Loss : 0.01065  Real : [7 8 9 8]  Pred : [7 8 9 8]
Index : 496  Loss : 0.03505  Real : [7 9 9 8]  Pred : [7 9 9 8]
Index : 497  Loss : 0.06497  Real : [7 7 7 9]  Pred : [7 7 7 9]
Index : 498  Loss : 0.05163  Real : [9 9

Index : 11  Loss : 2.24517  Real : [8 9 7 8]  Pred : [0 4 7 5]
Index : 12  Loss : 2.25230  Real : [8 7 8 7]  Pred : [1 2 2 7]
Index : 13  Loss : 1.68006  Real : [8 7 9 7]  Pred : [0 7 1 7]
Index : 14  Loss : 2.28233  Real : [8 8 8 8]  Pred : [2 8 0 0]
Index : 15  Loss : 1.77472  Real : [7 9 8 9]  Pred : [1 4 5 9]
Index : 16  Loss : 0.40964  Real : [9 7 8 7]  Pred : [7 1 0 7]
Index : 17  Loss : 0.26109  Real : [7 9 9 7]  Pred : [7 9 9 7]
Index : 18  Loss : 0.35232  Real : [7 8 7 9]  Pred : [7 8 7 9]
Index : 19  Loss : 0.31908  Real : [9 9 7 9]  Pred : [9 9 7 7]
Index : 20  Loss : 0.19816  Real : [7 7 7 8]  Pred : [7 7 7 8]
Index : 21  Loss : 0.28120  Real : [9 7 7 7]  Pred : [9 7 7 7]
Index : 22  Loss : 0.36804  Real : [7 8 9 7]  Pred : [7 8 4 7]
Index : 23  Loss : 0.08233  Real : [7 9 7 7]  Pred : [7 9 7 7]
Index : 24  Loss : 0.27266  Real : [7 8 9 8]  Pred : [7 8 8 8]
Index : 25  Loss : 0.08646  Real : [7 8 9 9]  Pred : [7 8 9 9]
Index : 26  Loss : 0.25691  Real : [9 7 9 7]  Pred : [9

Index : 141  Loss : 0.02934  Real : [8 8 8 8]  Pred : [8 8 8 8]
Index : 142  Loss : 0.08634  Real : [8 7 9 9]  Pred : [8 7 9 9]
Index : 143  Loss : 0.02282  Real : [9 8 7 7]  Pred : [9 8 7 7]
Index : 144  Loss : 0.06338  Real : [9 7 7 8]  Pred : [9 7 7 8]
Index : 145  Loss : 0.06570  Real : [9 7 7 9]  Pred : [9 7 7 9]
Index : 146  Loss : 0.18380  Real : [8 8 7 9]  Pred : [8 8 7 9]
Index : 147  Loss : 0.09505  Real : [7 8 7 9]  Pred : [7 8 7 9]
Index : 148  Loss : 0.03179  Real : [8 8 9 7]  Pred : [8 8 9 7]
Index : 149  Loss : 0.24360  Real : [7 8 9 9]  Pred : [7 8 9 9]
Index : 150  Loss : 0.30718  Real : [7 9 8 9]  Pred : [7 9 8 9]
Index : 151  Loss : 0.02968  Real : [7 9 9 7]  Pred : [7 9 9 7]
Index : 152  Loss : 0.03376  Real : [7 9 8 8]  Pred : [7 9 8 8]
Index : 153  Loss : 0.02434  Real : [7 9 7 9]  Pred : [7 9 7 9]
Index : 154  Loss : 0.05354  Real : [7 9 8 7]  Pred : [7 9 8 7]
Index : 155  Loss : 0.05944  Real : [9 7 9 7]  Pred : [9 7 9 7]
Index : 156  Loss : 0.52751  Real : [7 8

Index : 271  Loss : 0.02008  Real : [9 7 8 9]  Pred : [9 7 8 9]
Index : 272  Loss : 0.03366  Real : [7 9 7 7]  Pred : [7 9 7 7]
Index : 273  Loss : 0.07714  Real : [8 7 7 8]  Pred : [8 7 7 8]
Index : 274  Loss : 0.02988  Real : [8 9 7 7]  Pred : [8 9 7 7]
Index : 275  Loss : 0.18881  Real : [7 7 7 7]  Pred : [7 7 7 7]
Index : 276  Loss : 0.14290  Real : [9 7 7 7]  Pred : [9 7 7 7]
Index : 277  Loss : 0.19306  Real : [7 9 7 9]  Pred : [7 9 7 8]
Index : 278  Loss : 0.01038  Real : [7 9 8 7]  Pred : [7 9 8 7]
Index : 279  Loss : 0.03029  Real : [7 7 7 8]  Pred : [7 7 7 8]
Index : 280  Loss : 0.02636  Real : [7 7 7 7]  Pred : [7 7 7 7]
Index : 281  Loss : 0.25265  Real : [7 7 9 8]  Pred : [7 7 9 8]
Index : 282  Loss : 0.04207  Real : [8 9 8 7]  Pred : [8 9 8 7]
Index : 283  Loss : 0.05687  Real : [8 9 9 8]  Pred : [8 9 9 8]
Index : 284  Loss : 0.04428  Real : [9 9 7 7]  Pred : [9 9 7 7]
Index : 285  Loss : 0.06725  Real : [7 9 8 7]  Pred : [7 9 8 7]
Index : 286  Loss : 0.05519  Real : [9 8

Index : 401  Loss : 0.21721  Real : [9 7 7 9]  Pred : [9 7 7 8]
Index : 402  Loss : 0.01813  Real : [9 8 7 7]  Pred : [9 8 7 7]
Index : 403  Loss : 0.04230  Real : [8 9 9 7]  Pred : [8 9 9 7]
Index : 404  Loss : 0.01848  Real : [7 7 7 9]  Pred : [7 7 7 9]
Index : 405  Loss : 0.26226  Real : [8 8 8 9]  Pred : [8 8 8 8]
Index : 406  Loss : 0.06220  Real : [8 8 7 9]  Pred : [8 8 7 8]
Index : 407  Loss : 0.24268  Real : [7 7 7 8]  Pred : [7 7 7 8]
Index : 408  Loss : 0.05251  Real : [9 8 9 8]  Pred : [9 8 9 8]
Index : 409  Loss : 0.10172  Real : [7 8 8 9]  Pred : [7 8 8 9]
Index : 410  Loss : 0.09756  Real : [7 7 9 7]  Pred : [7 7 9 7]
Index : 411  Loss : 0.02975  Real : [8 7 8 7]  Pred : [8 7 8 7]
Index : 412  Loss : 0.01546  Real : [8 9 9 9]  Pred : [8 9 9 9]
Index : 413  Loss : 0.02330  Real : [8 9 9 7]  Pred : [8 9 9 7]
Index : 414  Loss : 0.22677  Real : [7 8 9 9]  Pred : [7 8 9 7]
Index : 415  Loss : 0.04456  Real : [8 9 7 9]  Pred : [8 9 7 9]
Index : 416  Loss : 0.06043  Real : [9 9

In [5]:
print("fsdg")

fsdg
